In [1]:
import os
import time
import datetime
import warnings
import random
from tqdm import tqdm

import numpy as np
from PIL import Image
import torch
import torchvision
import torchvision.transforms.functional as TF

from labels import labels

In [2]:
# 데이터 경로

origin_data_path = os.getcwd() + '/data/Original_data/'
labeled_data_path = os.getcwd() + '/data/Labeled_data/'

origin_data_list = os.listdir(origin_data_path) # x
labeled_data_list = os.listdir(labeled_data_path) # y

# 순서 랜덤하게 섞기
random.shuffle(origin_data_list)
random.shuffle(labeled_data_list)

# 학습용: 70%, 검증용: 10%, 테스트용 20% 분할  --> {train, val, test}_{x,y}_file: file명 저장 490, 70, 140
train_x_file = origin_data_list[:int(len(origin_data_list)*0.7)]
train_y_file = [file_name[:-4] + '_L.png' for file_name in train_x_file]

val_x_file = [file_name for file_name in origin_data_list if file_name not in train_x_file][:int(len(origin_data_list)*0.1)]
val_y_file = [file_name[:-4] + '_L.png' for file_name in val_x_file]

test_x_file = [file_name for file_name in origin_data_list if file_name not in (train_x_file + val_x_file)]
test_y_file = [file_name[:-4] + '_L.png' for file_name in test_x_file]

In [3]:
# color to label catId
color2label = { label.color   : label.id for label in labels}

In [4]:
train_y = []
for file_name in tqdm(train_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    train_y.append(ret)

100%|████████████████████████████████████████████████████████████████████████████████| 490/490 [31:53<00:00,  3.90s/it]


In [5]:
val_y = []
for file_name in tqdm(val_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    val_y.append(ret)

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [04:29<00:00,  3.85s/it]


In [6]:
test_y = []
for file_name in tqdm(test_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    test_y.append(ret)

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [08:57<00:00,  3.81s/it]


### original data {train/val/test}_x

In [59]:
train_x_o = []
for file_name in tqdm(train_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    train_x_o.append(np.array(image))

100%|████████████████████████████████████████████████████████████████████████████████| 490/490 [00:07<00:00, 63.01it/s]


In [60]:
val_x_o = []
for file_name in tqdm(val_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    val_x_o.append(np.array(image))

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:01<00:00, 61.86it/s]


In [61]:
test_x_o = []
for file_name in tqdm(test_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    test_x_o.append(np.array(image))

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [00:02<00:00, 63.11it/s]


In [62]:
train_x = np.array(train_x_o)
train_y = np.array(train_y)
val_x = np.array(val_x_o)
val_y = np.array(val_y)
test_x = np.array(test_x_o)
test_y = np.array(test_y)

# save dataset
np.savez('data_origin.npz', 
         train_x=train_x, train_y=train_y, 
         val_x=val_x, val_y=val_y, 
         test_x=test_x, test_y=test_y)

### notrm ver1

In [44]:
dataset_mean = [0.485, 0.456, 0.406]
dataset_std = [0.229, 0.224, 0.225]

In [45]:
test_x = []

for file_name in tqdm(test_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    image = TF.to_tensor(image)
        
    # Normalize
    image = TF.normalize(image, dataset_mean, dataset_std).permute(1, 2, 0)
    
    test_x.append(np.array(image))

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [00:02<00:00, 48.12it/s]


In [46]:
val_x = []
for file_name in tqdm(val_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    image = TF.to_tensor(image)

    # Normalize
    image = TF.normalize(image, dataset_mean, dataset_std).permute(1, 2, 0)
    
    val_x.append(np.array(image))

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:01<00:00, 47.84it/s]


In [47]:
train_x = []
train_x_jitter = []
for file_name in tqdm(train_x_file):
    image = np.array(Image.open(origin_data_path + file_name))   

    # From PIL to Tensor
    image = TF.to_tensor(image)
        
    # Normalize
    image = TF.normalize(image, dataset_mean, dataset_std).permute(1, 2, 0)
    
    train_x.append(np.array(image))

100%|████████████████████████████████████████████████████████████████████████████████| 490/490 [00:10<00:00, 47.01it/s]


In [48]:
train_x = np.array(train_x)
train_y = np.array(train_y)
val_x = np.array(val_x)
val_y = np.array(val_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

# save dataset
np.savez('data_norm_v1.npz', 
         train_x=train_x, train_y=train_y,
         val_x=val_x, val_y=val_y, 
         test_x=test_x, test_y=test_y)

### color jitter data (only train_x)

In [30]:
# color jitter
colorjitter_factor = 0.3

bf = np.random.uniform(1-colorjitter_factor,1+colorjitter_factor) # 명도
cf = np.random.uniform(1-colorjitter_factor,1+colorjitter_factor) # 대비
sf = np.random.uniform(1-colorjitter_factor,1+colorjitter_factor) # 채도
hf = np.random.uniform(-colorjitter_factor,+colorjitter_factor) # 색상

### norm ver2 (data - dataset_mean) / dataset_std

In [31]:
# compute dataset mean, std
x = []
for file_name in tqdm(train_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    x.append(image)
x=np.array(x)

dataset_mean = x.mean(axis=(0, 1, 2)) / 255  
dataset_std = x.std(axis=(0, 1, 2)) / 255    

100%|████████████████████████████████████████████████████████████████████████████████| 490/490 [00:06<00:00, 70.40it/s]


In [32]:
test_x = []

for file_name in tqdm(test_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    image = TF.to_tensor(image)
        
    # Normalize
    image = TF.normalize(image, dataset_mean, dataset_std).permute(1, 2, 0)
    
    test_x.append(np.array(image))

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [00:03<00:00, 46.97it/s]


In [33]:
val_x = []
for file_name in tqdm(val_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    image = TF.to_tensor(image)

    # Normalize
    image = TF.normalize(image, dataset_mean, dataset_std).permute(1, 2, 0)
    
    val_x.append(np.array(image))

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:01<00:00, 48.42it/s]


In [34]:
train_x = []
train_x_jitter = []
for file_name in tqdm(train_x_file):
    image = np.array(Image.open(origin_data_path + file_name))
    img_jitter = Image.open(origin_data_path + file_name)
    
    # color jitter
    img_jitter = TF.adjust_brightness(img_jitter, bf)
    img_jitter = TF.adjust_contrast(img_jitter, cf)
    img_jitter = TF.adjust_saturation(img_jitter, sf)
    img_jitter = TF.adjust_hue(img_jitter, hf)    

    # From PIL to Tensor
    img_jitter = TF.to_tensor(img_jitter)
    image = TF.to_tensor(image)
        
    # Normalize
    image = TF.normalize(image, dataset_mean, dataset_std).permute(1, 2, 0)
    img_jitter = TF.normalize(img_jitter, dataset_mean, dataset_std).permute(1, 2, 0)
    
    train_x.append(np.array(image))
    train_x_jitter.append(np.array(img_jitter))

100%|████████████████████████████████████████████████████████████████████████████████| 490/490 [00:43<00:00, 11.31it/s]


In [35]:
train_x = np.array(train_x)
train_y = np.array(train_y)
train_x_jitter = np.array(train_x_jitter)
val_x = np.array(val_x)
val_y = np.array(val_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [36]:
# save dataset
np.savez('data_norm_v2.npz', 
         train_x=train_x, train_y=train_y, 
         train_x_jitter=train_x_jitter,
         val_x=val_x, val_y=val_y, 
         test_x=test_x, test_y=test_y)